# LLM Logging 

## Logging Your Work 

### Why Logging Matters

* Reproducibility: What prompt did I use?
* Accountability: When did I run this?
* Version tracking: What changed since last run?

###  Example Task: Log a Simple LLM Prompt
- Save log files `logfile_YYYY-MM-DD.jsonl` to `logs` folder.
- Use `.jsonl` (JSON Lines) format for logging experiments, easy to parse later
    - Write
    ```python
    log_entry = {
        "timestamp": now.isoformat(),
        "model": llm_model,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "prompt": prompt,
        "output": output,
    }

    with open(output_file, "a") as f:
        f.write(json.dumps(log_entry) + "\n")
    ```

    - Read
    ```python
    import pandas as pd
    df = pd.read_json("logfile_2025-07-01.jsonl", lines=True)
    ```

### Lab 1: Generate marketing materials
- Track changes in model output over time or when prompts/models change.
- Modify to read prompt from file and add file to git 
    ```bash
    git add prompt.txt
    git commit -m "add prompt text file"
    ```
